# Финальное задание

## Предметная область: Игра Dota 2

[Dota 2](https://ru.wikipedia.org/wiki/Dota_2) — многопользовательская компьютерная игра жанра [MOBA](https://ru.wikipedia.org/wiki/MOBA). Игроки играют между собой матчи. В каждом матче участвует две команды, 5 человек в каждой. Одна команда играет за светлую сторону (The Radiant), другая — за тёмную (The Dire). Цель каждой команды — уничтожить главное здание базы противника (трон).

Существуют [разные режимы игры](http://dota2.gamepedia.com/Game_modes/ru), мы будем рассматривать режим [Captain's Mode](http://dota2.gamepedia.com/Game_modes/ru#Captain.27s_Mode), в формате которого происходит большая часть киберспортивных мероприятий по Dota 2.

### Как проходит матч

#### 1. Игроки выбирают героев

Всего в игре чуть более 100 различных героев (персонажей). В начале игры, команды в определенном порядке выбирают героев себе и запрещают выбирать определенных героев противнику (баны). Каждый игрок будет управлять одним героем, в рамках одного матча не может быть несколько одинаковых героев.  Герои различаются между собой своими характеристиками и способностями. От комбинации выбранных героев во многом зависит успех команды.

![](http://imgur.com/XFr4HYE.jpg)

#### 2. Основная часть

Игроки могут получать золото и опыт за убийство чужих героев или прочих юнитов. Накопленный опыт влияет на уровень героя, который в свою очередь позволяет улучшать способности. За накопленное золото игроки покупают предметы, которые улучшают характеристики героев или дают им новые способности.

После смерти герой отправляется в "таверну" и возрождается только по прошествии некоторого времени, таким образом команда на некоторое время теряет игрока, однако игрок может досрочно выкупить героя из таверны за определенную сумму золота.

В течение игры команды развивают своих героев, обороняют свою часть поля и нападают на вражескую.

![](http://imgur.com/5b0SlQb.jpg)

#### 3. Конец игры

Игра заканчивается, когда одна из команд разрушет определенное число "башен" противника и уничтожает трон.

![](http://imgur.com/Du79Kzf.jpg)

## Задача: предсказание победы по данным о первых 5 минутах игры

По первым 5 минутам игры предсказать, какая из команд победит: Radiant или Dire?

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from numpy import median
import time
import datetime

In [2]:
features = pd.read_csv('./features.csv', index_col='match_id')

features.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63


### Данные, которые нужно удалить
- Итог матча (данные поля отсутствуют в тестовой выборке, поскольку содержат информацию, выходящую за пределы первых 5 минут матча)
    - `duration`: длительность
    - `radiant_win`: 1, если победила команда Radiant, 0 — иначе
    - Состояние башен и барраков к концу матча (см. описание полей набора данных)
        - `tower_status_radiant`
        - `tower_status_dire`
        - `barracks_status_radiant`
        - `barracks_status_dire`

In [3]:
features_for_drop = ['duration', 'radiant_win', 'tower_status_radiant', 'tower_status_dire', \
                  'barracks_status_radiant', 'barracks_status_dire']
y = features['radiant_win'].values
features = features.drop(features_for_drop, axis = 1)
features.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0


Проверим выборку на наличие пропусков

In [4]:
rows = features.shape[0]
for (i , j) in enumerate(features.count()):
    if j != rows:
        print(features.columns[i])

first_blood_time
first_blood_team
first_blood_player1
first_blood_player2
radiant_bottle_time
radiant_courier_time
radiant_flying_courier_time
radiant_first_ward_time
dire_bottle_time
dire_courier_time
dire_flying_courier_time
dire_first_ward_time


In [5]:
features = features.fillna(0)

`radiant_win` - целевая переменная

### Подход 1: градиентный бустинг "в лоб"

In [6]:
kf = KFold(n_splits = 5, shuffle = True)
for count_trees in [10, 20, 30]: 
    print('Count trees:', count_trees)
    start_time = datetime.datetime.now()
    clf = GradientBoostingClassifier(n_estimators = count_trees)
    scores = []
    X = features.values
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        clf  = clf.fit(X_train, y_train)
        pred = clf.predict_proba(X_test)[:, 1]
        scores.append(roc_auc_score(y_test, pred))   
    print(median(scores))
    print('Time elapsed:', datetime.datetime.now() - start_time)

Count trees: 10
0.66247649665
Time elapsed: 0:00:39.802392
Count trees: 20
0.681830262549
Time elapsed: 0:01:17.429448
Count trees: 30
0.688082557448
Time elapsed: 0:01:35.587660


### Подход 2: логистическая регрессия

In [7]:
X = StandardScaler().fit_transform(features)
for C in [0.5, 1.0, 5, 10, 50, 100]:
    print("C =", C)
    start_time = datetime.datetime.now()
    clf = LogisticRegression(C = C)
    scores = []
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        clf  = clf.fit(X_train, y_train)
        pred = clf.predict_proba(X_test)[:, 1]
        scores.append(roc_auc_score(y_test, pred))   
    print(median(scores))
    print('Time elapsed:', datetime.datetime.now() - start_time)

C = 0.5
0.71650126421
Time elapsed: 0:00:10.815151
C = 1.0
0.714898132808
Time elapsed: 0:00:10.948613
C = 5
0.714692151224
Time elapsed: 0:00:11.120768
C = 10
0.718644896654
Time elapsed: 0:00:10.776769
C = 50
0.714888665434
Time elapsed: 0:00:10.383718
C = 100
0.716881797521
Time elapsed: 0:00:10.368553


Удаляем категориальных признаки: lobby_type и r1_hero, r2_hero, ..., r5_hero, d1_hero, d2_hero, ..., d5_hero. 

In [8]:
categorial = ['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', \
             'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']
X = features.drop(categorial, axis = 1)
X = StandardScaler().fit_transform(X)
for C in [0.5, 1.0, 5, 10, 50, 100]:
    print("C =", C)
    start_time = datetime.datetime.now()
    clf = LogisticRegression(C = C)
    scores = []
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        clf  = clf.fit(X_train, y_train)
        pred = clf.predict_proba(X_test)[:, 1]
        scores.append(roc_auc_score(y_test, pred))   
    print(median(scores))
    print('Time elapsed:', datetime.datetime.now() - start_time)

C = 0.5
0.716035590533
Time elapsed: 0:00:09.707648
C = 1.0
0.716879859226
Time elapsed: 0:00:09.361940
C = 5
0.713872388049
Time elapsed: 0:00:10.176357
C = 10
0.716532010925
Time elapsed: 0:00:11.365870
C = 50
0.715008378124
Time elapsed: 0:00:10.306107
C = 100
0.716710544575
Time elapsed: 0:00:11.024843


На предыдущем шаге мы исключили из выборки признаки rM_hero и dM_hero, которые показывают, какие именно герои играли за каждую команду. Это важные признаки — герои имеют разные характеристики, и некоторые из них выигрывают чаще, чем другие. Выясним из данных, сколько различных идентификаторов героев существует в данной игре:

In [9]:
max(max(features[i].unique()) for i in categorial[1:])

112

Воспользуемся подходом "мешок слов" для кодирования информации о героях. Пусть всего в игре имеет N различных героев. Сформируем N признаков, при этом i-й будет равен нулю, если i-й герой не участвовал в матче; единице, если i-й герой играл за команду Radiant; минус единице, если i-й герой играл за команду Dire.

In [10]:
# N — количество различных героев в выборке
N = 112
X_pick = np.zeros((features.shape[0], N))

for i, match_id in enumerate(features.index):
    for p in range(5):
        X_pick[i, features.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, features.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1
X_pick

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0., -1.]])

Кросс-валидация для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. 

In [11]:
X = features.drop(categorial, axis = 1)
X = np.concatenate((X, X_pick), axis = 1)
X = StandardScaler().fit_transform(X)
for C in [0.5, 1.0, 5, 10, 50, 100]:
    print("C =", C)
    start_time = datetime.datetime.now()
    clf = LogisticRegression(C = C)
    scores = []
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        clf  = clf.fit(X_train, y_train)
        pred = clf.predict_proba(X_test)[:, 1]
        scores.append(roc_auc_score(y_test, pred))   
    print(median(scores))
    print('Time elapsed:', datetime.datetime.now() - start_time)

C = 0.5
0.753316646236
Time elapsed: 0:00:21.849907
C = 1.0
0.751284878792
Time elapsed: 0:00:20.955479
C = 5
0.751807883749
Time elapsed: 0:00:21.632488
C = 10
0.751826768634
Time elapsed: 0:00:20.929741
C = 50
0.752715708664
Time elapsed: 0:00:19.917187
C = 100
0.752106979961
Time elapsed: 0:00:21.704251


Построем предсказания вероятностей победы команды Radiant для тестовой выборки с помощью лучшей из изученных моделей (лучшей с точки зрения AUC-ROC на кросс-валидации) - логистической регрессии (С = 5) 

In [12]:
X_test = pd.read_csv('./features_test.csv', index_col='match_id')
X_test = X_test.fillna(0)

X_pick = np.zeros((X_test.shape[0], N))
for i, match_id in enumerate(X_test.index):
    for p in range(5):
        X_pick[i, X_test.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, X_test.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

X_test = X_test.drop(categorial, axis = 1)
X_test = np.concatenate((X_test, X_pick), axis = 1)
X_test = StandardScaler().fit_transform(X_test)

clf = LogisticRegression(C = 5)
clf  = clf.fit(X, y)
pred = clf.predict_proba(X_test)[:, 1]
pred

array([ 0.82499248,  0.75854967,  0.18699254, ...,  0.23114843,
        0.6319573 ,  0.42598343])

In [13]:
print(min(pred), max(pred))

0.00868996533769 0.996487108697
